<a href="https://colab.research.google.com/github/stevenbowler/LamdaLabsTest/blob/master/colabs/AutoTrain_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title 🤗 AutoTrain LLM
#@markdown In order to use this colab
#@markdown - upload train.csv to a folder named `data/`
#@markdown - train.csv must contain a `text` column
#@markdown - choose a project name if you wish
#@markdown - change model if you wish, you can use most of the text-generation models from Hugging Face Hub
#@markdown - add huggingface information (token and repo_id) if you wish to push trained model to huggingface hub
#@markdown - update hyperparameters if you wish
#@markdown - click `Runtime > Run all` or run each cell individually

import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup > setup_logs.txt

Traceback (most recent call last):
  File "/usr/local/bin/autotrain", line 5, in <module>
    from autotrain.cli.autotrain import main
  File "/usr/local/lib/python3.10/dist-packages/autotrain/cli/autotrain.py", line 10, in <module>
    from .run_spacerunner import RunAutoTrainSpaceRunnerCommand
  File "/usr/local/lib/python3.10/dist-packages/autotrain/cli/run_spacerunner.py", line 3, in <module>
    from autotrain.backend import SpaceRunner
  File "/usr/local/lib/python3.10/dist-packages/autotrain/backend.py", line 11, in <module>
    from autotrain.dataset import AutoTrainDataset, AutoTrainDreamboothDataset
  File "/usr/local/lib/python3.10/dist-packages/autotrain/dataset.py", line 7, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.10/dist-packages/pandas/__init__.py", line 48, in <module>
    from pandas.core.api import (
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/api.py", line 27, in <module>
    from pandas.core.arrays import Categorical
  File 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp "/content/drive/MyDrive/ColabStuff/train.csv" "/content/data/train.csv"


cp: cannot create regular file '/content/data/train.csv': No such file or directory


In [3]:
import os

In [5]:
#@markdown ---
#@markdown #### Project Config
#@markdown Note: if you are using a restricted/private model, you need to enter your Hugging Face token in the next step.
project_name = 'FineTuneLLaMa2' # @param {type:"string"}
model_name = 'TinyPixel/Llama-2-7B-bf16-sharded' # @param {type:"string"}

#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = True # @param ["False", "True"] {type:"raw"}
hf_token = "" #@param {type:"string"}
repo_id = "stevenbowler/MedChatBot4" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 2e-4 # @param {type:"number"}
num_epochs = 1 #@param {type:"number"}
batch_size = 4 # @param {type:"slider", min:1, max:32, step:1}
block_size = 512 # @param {type:"number"}
trainer = "sft" # @param ["default", "sft"] {type:"raw"}
warmup_ratio = 0.1 # @param {type:"number"}
weight_decay = 0.01 # @param {type:"number"}
gradient_accumulation = 4 # @param {type:"number"}
use_fp16 = True # @param ["False", "True"] {type:"raw"}
use_peft = True # @param ["False", "True"] {type:"raw"}
use_int4 = True # @param ["False", "True"] {type:"raw"}
lora_r = 16 #@param {type:"number"}
lora_alpha = 32 #@param {type:"number"}
lora_dropout = 0.05 #@param {type:"number"}

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["USE_PEFT"] = str(use_peft)
os.environ["USE_INT4"] = str(use_int4)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)


In [ ]:
push_to_hub

True

In [ ]:
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256'

In [16]:
import getpass

hf_token=getpass.getpass('HuggingFace Write Key: ')

HuggingFace Write Key: ··········


In [ ]:
# !autotrain setup --update-torch   # recommended for colab

In [ ]:
#failed due to last 4 variables not being set, syntax
!autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path data/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_PEFT" == "True" ]] && echo "--use-peft" ) \
$( [[ "$USE_INT4" == "True" ]] && echo "--use-int4" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

/usr/bin/sh: 1: [[: not found
/usr/bin/sh: 1: [[: not found
/usr/bin/sh: 1: [[: not found
/usr/bin/sh: 1: [[: not found
--------------------------------------------------------------------------
detected:

  Local host:            129-159-39-145
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4126

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found to 0.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:   

In [7]:
!pip install huggingface_hub

In [8]:
# connect to huggingface
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path data/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--fp16 \
--use-peft \
--use-int4 \
--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}

--------------------------------------------------------------------------
detected:

  Local host:            129-159-39-145
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4126

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found to 0.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           129-159-39-145
  Local device:         mlx5_0
  Local port:           1
  CPCs attempted:       udcm
-----------

In [9]:
adapter_model_name = "stevenbowler/MedChatBot4"
model_name = 'TinyPixel/Llama-2-7B-bf16-sharded'


In [10]:
!pip install transformers

In [11]:
from transformers import AutoModelForCausalLM

adapter_model = AutoModelForCausalLM.from_pretrained(adapter_model_name)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [26]:

from torch import cuda, bfloat16
import transformers

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_quant_type='nf4',
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    gradient_checkpointing=True,
    use_auth_token=hf_token
)

model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      config=model_config,
      quantization_config=bnb_config,
      device_map='auto',
      use_auth_token=hf_token
      )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1015: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [27]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, adapter_model_name)

In [28]:
model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:68: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [29]:
new_model_name = "stevenbowler/MedChatBotAdapted"

model.push_to_hub(new_model_name, use_auth_token=True, max_shard_size="5GB")


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1938: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/stevenbowler/MedChatBotAdapted/commit/0a9bbea3ab15e6ae15535275f0654f5e805ded13', commit_message='Upload LlamaForCausalLM', commit_description='', oid='0a9bbea3ab15e6ae15535275f0654f5e805ded13', pr_url=None, pr_revision=None, pr_num=None)